# **Setup**

In [ ]:
!pip install -qq langchain
!pip install -qq langchain-community
!pip install -qq langchain-google-genai

In [2]:
import os

In [3]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [4]:
os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3)

# **1) Static prompts (plain templates, no variables)**

Use when the instruction never changes.

In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are concise and factual."),
    ("human", "List three practical uses of Retrieval-Augmented Generation for e-commerce.")
])

chain = prompt | llm | StrOutputParser()
print(chain.invoke({}))


1. **Improved Product Descriptions:** RAG can enhance product descriptions by incorporating information from various sources like specifications, reviews, and competitor offerings.

2. **Personalized Recommendations:** RAG can generate more relevant product recommendations by accessing and synthesizing user data, purchase history, and product information.

3. **Enhanced Customer Service:** RAG can power chatbots that provide more accurate and comprehensive answers to customer queries by accessing a knowledge base of product information and FAQs.


# **2) Dynamic prompts (variables + partials)**

1. Inject runtime values into the template; reuse with different inputs.
2. Also handy: partials to “pre-fill” some variables.

In [8]:
from langchain_core.prompts import PromptTemplate

base = PromptTemplate.from_template(
    "Write a {tone} {words}-word product blurb.\n"
    "Name: {name}\n"
    "Features: {features}\n"
)

# Pre-fill some variables once (partials can stack)
prompt = base.partial(tone="engaging").partial(words="60")

chain = prompt | llm | StrOutputParser()
print(chain.invoke({
    "name": "AquaSense Smart Bottle",
    "features": "temperature display, hydration reminders, stainless steel"
}))


Stay perfectly hydrated with AquaSense!  This sleek stainless steel smart bottle boasts a built-in temperature display and smart hydration reminders, ensuring you drink enough throughout the day.  Never guess your water temperature again – AquaSense keeps you informed and refreshed.  Upgrade your hydration game today!


# **3) Role-based (chat) prompts (system/human + message placeholders)**

1. Use chat roles to separate “behavior” (system) from “request” (human).
2. Add MessagesPlaceholder now and you can plug in memory later with the same prompt.

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a senior TypeScript mentor. Answer with precise code and a short note."),
    MessagesPlaceholder("history"),     # optional, for chat memory later
    ("human", "{question}")
])

chain = prompt | llm | StrOutputParser()

# no history yet; just pass an empty list
print(chain.invoke({
    "history": [],
    "question": "Explain discriminated unions with a minimal example."
}))


```typescript
type Shape = 
  | { kind: "circle"; radius: number }
  | { kind: "square"; sideLength: number };

function getArea(shape: Shape) {
  switch (shape.kind) {
    case "circle": return Math.PI * shape.radius ** 2;
    case "square": return shape.sideLength ** 2;
  }
}

```

**Note:** The `kind` property acts as a discriminator, allowing TypeScript to narrow down the type based on its value, ensuring type safety within the `getArea` function.


# **4) Few-shot prompts (teach by examples)**

Prime the model with several (Q,A) examples before your real query.

## **4a) Text few-shot**

In [10]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser

examples = [
    {"question": "Convert 30°C to Fahrenheit.", "answer": "86°F (30×9/5 + 32)."},
    {"question": "Plural of 'cactus'?", "answer": "'cacti' (also 'cactuses' in informal use)."},
]

example_prompt = PromptTemplate.from_template(
    "Q: {question}\nA: {answer}"
)

few_shot = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="You are a concise tutor. Use the examples to guide your answer.\n",
    suffix="Q: {input}\nA:",
    input_variables=["input"],
)

chain = few_shot | llm | StrOutputParser()
print(chain.invoke({"input": "Give the formula and one-line intuition for the area of a circle."}))


A = πr²;  Area equals pi times the radius squared.


## **4b) Chat few-shot**

In [11]:
from langchain_core.prompts import ChatPromptTemplate

chat_example = ChatPromptTemplate.from_messages([
    ("human", "{question}"),
    ("ai", "{answer}")
])

# format examples into chat messages
example_msgs = []
for ex in examples:
    example_msgs += chat_example.format_messages(**ex)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer succinctly and correctly."),
    *example_msgs,
    ("human", "{input}")
])

chain = prompt | llm | StrOutputParser()
print(chain.invoke({"input": "When should I use a hashmap vs an array?"}))


Use a HashMap when you need fast lookups by key, and an array when you need fast access by index.


# **format instructions pattern (for JSON outputs)**

In [13]:
from langchain.output_parsers.structured import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate

schemas = [
    ResponseSchema(name="title", description="Short title"),
    ResponseSchema(name="summary", description="2-3 sentence summary"),
    ResponseSchema(name="tags", description="Array of 3-5 tags"),
]
parser = StructuredOutputParser.from_response_schemas(schemas)

prompt = PromptTemplate.from_template(
    "Extract structured info from the text.\n"
    "{format_instructions}\n"
    "Text:\n{text}"
).partial(format_instructions=parser.get_format_instructions())

chain = prompt | llm | parser
print(chain.invoke({"text": "LangChain lets you build LLM apps via prompts, chains, tools, and memory."}))


{'title': 'LangChain', 'summary': 'LangChain is a framework for building applications powered by large language models (LLMs).  It simplifies development by providing tools for managing prompts, chains of operations, external tools, and memory.', 'tags': ['LLM', 'Prompt Engineering', 'Chain', 'Tools', 'Memory']}
